### Module Import and Auto-Reload

In [2]:
# Make sure autoreload is properly set up for auto reload the module if changes
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))
print(parent_dir)

/home/ct-admin/Documents/Langgraph/00_AudioProject/02_Call_Center_AI_Agent


### Test

In [18]:
# ./example_usage.py
"""
Example of how to use the optimized call center system.
Shows integration between call_scripts.py, data_parameter_builder.py, and prompts.py
"""

from src.Agents.call_center_agent.call_scripts import ScriptType, CallStep
from src.Agents.call_center_agent.data_parameter_builder import (
    get_client_data,
    prepare_parameters, 
    analyze_client_behavior,
    ConversationState,
    VerificationStatus
)
from src.Agents.call_center_agent.prompts import get_step_prompt

def example_call_flow():
    """Example of a complete call flow using the optimized system."""
    
    # 1. Initialize call parameters
    user_id = "83906"
    agent_name = "Sarah Mitchell"
    script_type = "ratio_1_inflow"  # Standard first missed payment
    
    # 2. Load client data once at the beginning of the call
    print("Loading client data...")
    client_data = get_client_data(user_id)
    print(f"Loaded data for: {client_data.get('profile', {}).get('client_info', {}).get('client_full_name', 'Client')}")
    
    # 3. Initialize conversation state
    state = ConversationState()
    
    # 4. STEP 1: Introduction
    print("=== STEP 1: INTRODUCTION ===")
    parameters = prepare_parameters(
        client_data=client_data,
        current_step=CallStep.INTRODUCTION.value,
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    introduction_prompt = get_step_prompt("introduction", parameters)
    print("Agent prompt:", introduction_prompt if len(introduction_prompt) > 200 else introduction_prompt)

    # 5. STEP 2: Name Verification  
    print("\n=== STEP 2: NAME VERIFICATION ===")
    
    # Update state - client confirmed identity
    state.name_verification_status = VerificationStatus.VERIFIED.value
    parameters = prepare_parameters(
        client_data=client_data,
        current_step=CallStep.NAME_VERIFICATION.value,
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    name_verification_prompt = get_step_prompt("name_verification", parameters)
    print("Agent prompt:", name_verification_prompt if len(name_verification_prompt) > 200 else name_verification_prompt)
    print(f"Third party message: {parameters.get('third_party_message', 'N/A')}")
    
    # 6. STEP 3: Details Verification
    print("\n=== STEP 3: DETAILS VERIFICATION ===")
    
    # Update state - start details verification
    state.details_verification_attempts = 1
    state.add_matched_field("id_number")  # Client provided ID
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="details_verification",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    details_prompt = get_step_prompt("details_verification", parameters)
    print("Agent prompt:", details_prompt if len(details_prompt) > 200 else details_prompt)
    print(f"Details verification script: {parameters.get('details_verification_script', 'N/A')}")
    
    # 7. STEP 4: Reason for Call
    print("\n=== STEP 4: REASON FOR CALL ===")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="reason_for_call",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    reason_prompt = get_step_prompt("reason_for_call", parameters)
    print("Agent prompt:", reason_prompt if len(reason_prompt) > 200 else reason_prompt)
    print(f"Formatted script: {parameters.get('script_content', 'N/A')}")
    
    # 8. STEP 5: Negotiation (with objection handling)
    print("\n=== STEP 5: NEGOTIATION ===")
    
    # Simulate client objection
    state.add_objection("no_money")
    state.update_emotional_state("defensive")
    state.update_payment_willingness("low")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="negotiation",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    negotiation_prompt = get_step_prompt("negotiation", parameters)
    print("Agent prompt:", negotiation_prompt if len(negotiation_prompt) > 200 else negotiation_prompt)
    
    # Show how behavioral intelligence is integrated
    print("\nBehavioral Intelligence:")
    print(f"- Emotional State: {state.emotional_state}")
    print(f"- Objections: {state.objections_raised}")
    print(f"- Payment Willingness: {state.payment_willingness}")

    # Access flattened tactical guidance parameters
    tactical_guidance = {
        "recommended_approach": parameters.get("tactical_guidance[recommended_approach]", "N/A"),
        "urgency_level": parameters.get("tactical_guidance[urgency_level]", "N/A"),
        "success_probability": parameters.get("tactical_guidance[success_probability]", "N/A"),
        "key_motivators": parameters.get("tactical_guidance[key_motivators]", "N/A")
    }
    print(f"- Tactical Guidance: {tactical_guidance}")
    print(f"- Consequences Script: {parameters.get('consequences_script', 'N/A')}")
    print(f"- Benefits Script: {parameters.get('benefits_script', 'N/A')}")
    
    # 9. STEP 6: Promise to Pay
    print("\n=== STEP 6: PROMISE TO PAY ===")
    
    # Client becomes more cooperative
    state.update_emotional_state("cooperative")
    state.update_payment_willingness("medium")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="promise_to_pay",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    ptp_prompt = get_step_prompt("promise_to_pay", parameters)
    print("Agent prompt:", ptp_prompt if len(ptp_prompt) > 200 else ptp_prompt)
    
    # 10. STEP 7: DebiCheck Setup
    print("\n=== STEP 7: DEBICHECK SETUP ===")
    
    # Client agrees to DebiCheck
    state_dict = state.to_dict()
    state_dict["payment_method"] = "DebiCheck"
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="debicheck_setup",
        state=state_dict,
        script_type=script_type,
        agent_name=agent_name
    )
    
    debicheck_prompt = get_step_prompt("debicheck_setup", parameters)
    print("Agent prompt:", debicheck_prompt if len(debicheck_prompt) > 200 else debicheck_prompt)
    print(f"Amount with fee: {parameters.get('amount_with_fee', 'N/A')}")
    
    # 11. STEP 8: Payment Portal (Alternative flow)
    print("\n=== STEP 8: PAYMENT PORTAL (Alternative) ===")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="payment_portal",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    portal_prompt = get_step_prompt("payment_portal", parameters)
    print("Agent prompt:", portal_prompt if len(portal_prompt) > 200 else portal_prompt)
    
    # 12. STEP 9: Subscription Reminder
    print("\n=== STEP 9: SUBSCRIPTION REMINDER ===")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="subscription_reminder",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    subscription_prompt = get_step_prompt("subscription_reminder", parameters)
    print("Agent prompt:", subscription_prompt if len(subscription_prompt) > 200 else subscription_prompt)
    print(f"Outstanding vs Subscription: {parameters.get('outstanding_amount', 'N/A')} vs {parameters.get('subscription_amount', 'N/A')}")
    
    # 13. STEP 10: Client Details Update
    print("\n=== STEP 10: CLIENT DETAILS UPDATE ===")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="client_details_update",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    details_update_prompt = get_step_prompt("client_details_update", parameters)
    print("Agent prompt:", details_update_prompt if len(details_update_prompt) > 200 else details_update_prompt)
    
    # 14. STEP 11: Referrals
    print("\n=== STEP 11: REFERRALS ===")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="referrals",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    referrals_prompt = get_step_prompt("referrals", parameters)
    print("Agent prompt:", referrals_prompt if len(referrals_prompt) > 200 else referrals_prompt)
    
    # 15. STEP 12: Further Assistance
    print("\n=== STEP 12: FURTHER ASSISTANCE ===")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="further_assistance",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    assistance_prompt = get_step_prompt("further_assistance", parameters)
    print("Agent prompt:", assistance_prompt if len(assistance_prompt) > 200 else assistance_prompt)
    
    # 16. STEP 13: Cancellation (Alternative flow)
    print("\n=== STEP 13: CANCELLATION (Alternative) ===")
    
    # Simulate cancellation request
    state_dict = state.to_dict()
    state_dict["cancellation_fee"] = "R 150.00"
    state_dict["total_balance"] = parameters.get("outstanding_amount", "R 0.00")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="cancellation",
        state=state_dict,
        script_type=script_type,
        agent_name=agent_name
    )
    
    cancellation_prompt = get_step_prompt("cancellation", parameters)
    print("Agent prompt:", cancellation_prompt if len(cancellation_prompt) > 200 else cancellation_prompt)
    print(f"Cancellation fee: {parameters.get('cancellation_fee', 'N/A')}")
    
    # 17. STEP 14: Escalation (Alternative flow)
    print("\n=== STEP 14: ESCALATION (Alternative) ===")
    
    # Simulate escalation scenario
    state_dict = state.to_dict()
    state_dict["department"] = "Supervisor"
    state_dict["response_time"] = "24 hours"
    state_dict["ticket_number"] = "ESC123456"
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="escalation",
        state=state_dict,
        script_type=script_type,
        agent_name=agent_name
    )
    
    escalation_prompt = get_step_prompt("escalation", parameters)
    print("Agent prompt:", escalation_prompt if len(escalation_prompt) > 200 else escalation_prompt)
    print(f"Ticket number: {parameters.get('ticket_number', 'N/A')}")
    
    # 18. STEP 15: Closing
    print("\n=== STEP 15: CLOSING ===")
    
    # Set outcome summary for closing
    state_dict = state.to_dict()
    state_dict["outcome_summary"] = "payment arrangement confirmed via DebiCheck"
    state_dict["payment_method"] = "DebiCheck"
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="closing",
        state=state_dict,
        script_type=script_type,
        agent_name=agent_name
    )
    
    closing_prompt = get_step_prompt("closing", parameters)
    print("Agent prompt:", closing_prompt if len(closing_prompt) > 200 else closing_prompt)
    
    # 19. STEP 16: Query Resolution (Example)
    print("\n=== STEP 16: QUERY RESOLUTION (Example) ===")
    
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="query_resolution",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    query_prompt = get_step_prompt("query_resolution", parameters)
    print("Agent prompt:", query_prompt if len(query_prompt) > 200 else query_prompt)
    
    print("\n=== CALL COMPLETED ===")
    print("Final State Summary:")
    print(f"- Verification Status: {state.details_verification_status}")
    print(f"- Objections Handled: {len(state.objections_raised)}")
    print(f"- Final Emotional State: {state.emotional_state}")
    print(f"- Payment Arrangement: Secured via DebiCheck")
    print(f"- Steps Completed: 16 different call steps tested")
    
    print("\nAll Available Steps Tested:")
    available_steps = [
        "introduction", "name_verification", "details_verification", "reason_for_call",
        "negotiation", "promise_to_pay", "debicheck_setup", "payment_portal", 
        "subscription_reminder", "client_details_update", "referrals", "further_assistance",
        "cancellation", "escalation", "closing", "query_resolution"
    ]
    for i, step in enumerate(available_steps, 1):
        print(f"  {i:2d}. {step}")

def example_script_variations():
    """Example showing different script types for different scenarios."""
    
    print("\n=== SCRIPT VARIATIONS EXAMPLE ===")
    
    user_id = "83906"
    agent_name = "Sarah Mitchell"
    
    # Load client data once
    client_data = get_client_data(user_id)
    
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    # Different script types for different scenarios
    scenarios = [
        ("ratio_1_inflow", "Standard first missed payment"),
        ("ratio_1_failed_ptp", "Failed promise to pay"),
        ("ratio_2_3_inflow", "2-3 months overdue"),
        ("pre_legal_120_plus", "Pre-legal 120+ days"),
        ("pre_legal_150_plus", "Legal action - attorney involved")
    ]
    
    for script_type, description in scenarios:
        print(f"\n--- {description.upper()} ---")
        
        parameters = prepare_parameters(
            client_data=client_data,
            current_step="reason_for_call",
            state=state.to_dict(),
            script_type=script_type,
            agent_name=agent_name
        )
        
        # Show script content differences
        script_content = parameters.get("script_content", "")
        print(f"Script: {script_content}")
        
        # Show tactical guidance differences  
        tactical = parameters.get("tactical_guidance", {})
        print(f"Urgency Level: {tactical.get('urgency_level')}")
        print(f"Key Motivators: {tactical.get('key_motivators')}")

def example_objection_handling():
    """Example of dynamic objection handling."""
    
    print("\n=== OBJECTION HANDLING EXAMPLE ===")
    
    from src.Agents.call_center_agent.call_scripts import ScriptManager, ScriptType
    
    script_type = ScriptType.RATIO_1_INFLOW
    
    # Show available objection responses
    common_objections = [
        "no_money",
        "dispute_amount", 
        "will_pay_later",
        "already_paid",
        "cant_afford"
    ]
    
    print("Available objection responses:")
    for objection in common_objections:
        response = ScriptManager.get_objection_response(script_type, objection)
        print(f"- {objection}: {response}")
    
    # Show emotional state responses
    print("\nEmotional state responses:")
    emotional_states = ["angry", "confused", "defensive", "dismissive"]
    for state in emotional_states:
        response = ScriptManager.get_emotional_response(script_type, state)
        print(f"- {state}: {response}")

def example_behavioral_analysis():
    """Example of behavioral analysis integration."""
    
    print("\n=== BEHAVIORAL ANALYSIS EXAMPLE ===")
    
    # Mock client data with payment history
    client_data = {
        "account_aging": {"xbalance": "650.00"},
        "payment_history": [
            {"status": "successful", "amount": "199.00"},
            {"status": "failed", "amount": "199.00"},
            {"status": "successful", "amount": "199.00"},
        ],
        "failed_payments": [
            {"status": "failed", "amount": "199.00"},
        ],
        "subscription": {"subscription_amount": "199.00"},
        "billing_analysis": {"balance_60_days": "350.00"},
        "account_overview": {"account_status": "Overdue"}
    }
    
    analysis = analyze_client_behavior(client_data)
    
    print("Behavioral Analysis Results:")
    for key, value in analysis.items():
        print(f"- {key}: {value}")
    
    print("\nThis analysis automatically influences:")
    print("- Prompt tactical guidance")
    print("- Objection predictions")
    print("- Recommended approach")
    print("- Success probability assessment")

def example_real_time_adaptation():
    """Example of how the system adapts during a live call."""
    
    print("\n=== REAL-TIME ADAPTATION EXAMPLE ===")
    
    user_id = "83906"
    agent_name = "Sarah Mitchell"
    script_type = "ratio_1_inflow"
    
    # Load client data once
    client_data = get_client_data(user_id)
    
    # Initial state
    state = ConversationState()
    print("Initial state: neutral, unknown payment willingness")
    
    # Call progresses and client becomes defensive
    state.update_emotional_state("defensive")
    state.add_objection("dispute_amount")
    
    print("After objection: defensive, dispute amount")
    
    # Generate adapted prompt
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="negotiation", 
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    print("System automatically provides:")
    
    # Safe access to emotional responses
    emotional_responses = parameters.get('emotional_responses', {})
    if isinstance(emotional_responses, dict):
        emotional_guidance = emotional_responses.get('defensive', 'Not available')
    else:
        # If it's already a formatted string, parse it or use as-is
        emotional_guidance = emotional_responses if emotional_responses else 'Not available'
    print(f"- Emotional response guidance: {emotional_guidance}")
    
    # Safe access to objection responses
    objection_responses = parameters.get('objection_responses', {})
    if isinstance(objection_responses, dict):
        objection_guidance = objection_responses.get('dispute_amount', 'Not available')
    else:
        # If it's already a formatted string, parse it or use as-is
        objection_guidance = objection_responses if objection_responses else 'Not available'
    print(f"- Objection response: {objection_guidance}")
    
    # Safe access to tactical guidance (flattened format)
    tactical_approach = parameters.get('tactical_guidance[recommended_approach]', 'Not available')
    print(f"- Tactical approach: {tactical_approach}")
    
    # Client becomes cooperative
    state.update_emotional_state("cooperative")
    state.update_payment_willingness("high")
    
    print("\nAfter rapport building: cooperative, high payment willingness")
    
    # System adapts approach
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="promise_to_pay",
        state=state.to_dict(), 
        script_type=script_type,
        agent_name=agent_name
    )
    
    print("System now recommends:")
    print(f"- Direct payment request approach")
    
    # Safe access to success probability (flattened format)
    success_probability = parameters.get('tactical_guidance[success_probability]', 'Not available')
    print(f"- Success probability: {success_probability}")


def example_data_separation_benefits():
    """Example showing benefits of separating data loading from parameter building."""
    
    print("\n=== DATA SEPARATION BENEFITS EXAMPLE ===")
    
    user_id = "83906"
    agent_name = "Sarah Mitchell"
    script_type = "ratio_1_inflow"
    
    # Load client data once
    print("1. Loading client data once...")
    client_data = get_client_data(user_id)
    
    # Build parameters for multiple steps without reloading data
    state = ConversationState()
    steps = ["introduction", "name_verification", "reason_for_call", "negotiation"]
    
    print("2. Building parameters for multiple steps (no additional database calls)...")
    for step in steps:
        print(f"   - Building parameters for {step}")
        parameters = prepare_parameters(
            client_data=client_data,
            current_step=step,
            state=state.to_dict(),
            script_type=script_type,
            agent_name=agent_name
        )
        # In a real scenario, you'd use these parameters to generate prompts
    
    print("3. Benefits:")
    print("   - Single database call for entire conversation")
    print("   - Faster parameter building for subsequent steps")
    print("   - Better testability with mock data")
    print("   - Cleaner separation of concerns")
    
    # Example of modifying client data before building parameters
    print("4. Example: Modifying client data before parameter building...")
    
    # Simulate updating outstanding amount during call
    original_amount = client_data.get("account_aging", {}).get("xbalance", "0")
    print(f"   - Original amount: R {original_amount}")
    
    # Update the amount (e.g., partial payment received)
    if "account_aging" not in client_data:
        client_data["account_aging"] = {}
    client_data["account_aging"]["xbalance"] = "150.00"  # Reduced amount
    
    # Build parameters with updated data
    parameters = prepare_parameters(
        client_data=client_data,
        current_step="reason_for_call",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
    print(f"   - Updated amount in script: {parameters.get('outstanding_amount', 'N/A')}")
    print("   - Script content now reflects updated amount without database reload")

if __name__ == "__main__":
    # Run all examples
    print("=" * 60)
    print("CALL CENTER SYSTEM EXAMPLES")
    print("=" * 60)
    

    example_call_flow()
    example_script_variations()
    example_objection_handling()
    example_behavioral_analysis()
    example_real_time_adaptation()
    example_data_separation_benefits()
    
    print("\n" + "=" * 60)
    print("ALL EXAMPLES COMPLETED SUCCESSFULLY")
    print("=" * 60)
        


2025-05-27 13:29:53,576 - INFO - Using cached data for user_id: 83906
2025-05-27 13:29:53,576 - INFO - Built parameters for user_id: 83906, step: introduction
2025-05-27 13:29:53,576 - INFO - Built parameters for user_id: 83906, step: name_verification
2025-05-27 13:29:53,577 - INFO - Verification field matched: id_number
2025-05-27 13:29:53,577 - INFO - Built parameters for user_id: 83906, step: details_verification
2025-05-27 13:29:53,577 - INFO - Built parameters for user_id: 83906, step: reason_for_call
2025-05-27 13:29:53,577 - INFO - Objection added: no_money
2025-05-27 13:29:53,577 - INFO - Emotional state updated to: defensive
2025-05-27 13:29:53,577 - INFO - Payment willingness updated to: low
2025-05-27 13:29:53,578 - INFO - Built parameters for user_id: 83906, step: negotiation
2025-05-27 13:29:53,578 - WARNING - Error formatting prompt for negotiation: 'behavioral_analysis'
2025-05-27 13:29:53,578 - INFO - Emotional state updated to: cooperative
2025-05-27 13:29:53,578 - IN

CALL CENTER SYSTEM EXAMPLES
Loading client data...
Loaded data for: Linda Oelofse
=== STEP 1: INTRODUCTION ===
Agent prompt: 
<role>
You are Sarah Mitchell, a professional debt collection specialist at Cartrack's Accounts Department.
</role>

<context>
- Making OUTBOUND call regarding overdue account
- CRITICAL: Cannot discuss ANY account details until client identity is FULLY verified
- Client may be defensive, evasive, or emotional - remain professional but assertive
- Current step: introduction
</context>

<communication_style>
- Professional, confident, and solution-focused
- Concise responses (max 30 words unless detailed explanation needed)
- Acknowledge emotions but redirect to resolution
- Use "I understand" to validate, then guide to next step
</communication_style>

<restrictions>
- NO account details or amounts until verification complete
- NO discussing reason for call until identity confirmed
- Stay focused but adapt tone to client response
</restrictions>

<task>
Create p

In [6]:
user_id = "83906"
agent_name = "Sarah Mitchell"
script_type = "ratio_1_inflow"  # Standard first missed payment
client_data = get_client_data(user_id)


2025-05-27 13:11:42,719 - INFO - Using cached data for user_id: 83906


In [12]:
state = ConversationState()
state.add_objection("no_money")
state.update_emotional_state("defensive")
state.update_payment_willingness("low")
parameters = prepare_parameters(
        client_data=client_data,
        current_step="negotiation",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
reason_prompt = get_step_prompt("reason_for_call", parameters)
print(reason_prompt)

2025-05-27 13:13:03,915 - INFO - Objection added: no_money
2025-05-27 13:13:03,915 - INFO - Emotional state updated to: defensive
2025-05-27 13:13:03,916 - INFO - Payment willingness updated to: low
2025-05-27 13:13:03,916 - INFO - Built parameters for user_id: 83906, step: negotiation



<role>
You are Sarah Mitchell, a professional debt collection specialist at Cartrack's Accounts Department.
</role>

<context>
- Making OUTBOUND call regarding overdue account
- CRITICAL: Cannot discuss ANY account details until client identity is FULLY verified
- Client may be defensive, evasive, or emotional - remain professional but assertive
- Current step: negotiation
</context>

<communication_style>
- Professional, confident, and solution-focused
- Concise responses (max 30 words unless detailed explanation needed)
- Acknowledge emotions but redirect to resolution
- Use "I understand" to validate, then guide to next step
</communication_style>

<restrictions>
- NO account details or amounts until verification complete
- NO discussing reason for call until identity confirmed
- Stay focused but adapt tone to client response
</restrictions>

<task>
Clearly communicate account status and required payment amount.
</task>

<script_foundation>
Core message: ""
</script_foundation>

<c

In [ ]:
example_call_flow()

In [ ]:
example_script_variations()

In [ ]:
example_objection_handling()

In [16]:
example_behavioral_analysis()


=== BEHAVIORAL ANALYSIS EXAMPLE ===
Behavioral Analysis Results:
- days_overdue: 0
- payment_reliability: high
- likely_objections: ['dispute_amount', 'no_money', 'will_pay_later']
- optimal_approach: friendly_reminder
- risk_level: medium
- success_probability: high

This analysis automatically influences:
- Prompt tactical guidance
- Objection predictions
- Recommended approach
- Success probability assessment


In [17]:
example_real_time_adaptation()

2025-05-27 12:03:32,862 - INFO - Emotional state updated to: defensive
2025-05-27 12:03:32,863 - INFO - Objection added: dispute_amount
2025-05-27 12:03:32,864 - INFO - Using cached data for user_id: 83906
2025-05-27 12:03:32,864 - INFO - Built parameters for user_id: 83906, step: negotiation
2025-05-27 12:03:32,864 - INFO - Emotional state updated to: cooperative
2025-05-27 12:03:32,865 - INFO - Payment willingness updated to: high
2025-05-27 12:03:32,865 - INFO - Using cached data for user_id: 83906
2025-05-27 12:03:32,865 - INFO - Built parameters for user_id: 83906, step: promise_to_pay



=== REAL-TIME ADAPTATION EXAMPLE ===
Initial state: neutral, unknown payment willingness
After objection: defensive, dispute amount
System automatically provides:
- Emotional response guidance: I'm here to help, not judge. Let's focus on keeping your vehicle protection active.
- Objection response: I can verify the breakdown while we arrange payment to avoid service interruption. What specifically concerns you?
- Tactical approach: friendly_reminder

After rapport building: cooperative, high payment willingness
System now recommends:
- Direct payment request approach
- Success probability: medium
